# Setup

In [1]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib qt
# matplotlib.backend
# matplotlib.validate_backend("GTK3Ag")

In [2]:
import numpy as np
import pickle
from tqdm.auto import tqdm as progressbar
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import cv2.aruco as aruco
import common as co
from os.path import join
from calib import calibrate_charuco_local, load_board, load_coefficients, save_coefficients

## Load calib

In [3]:
data_path="/home/freitas/TCC/ender-laser-scanner/pics/scan1_raspi/"

In [4]:
import json

# Calib params
with open("/home/freitas/TCC/ender-laser-scanner/calib_data/v1_raspi/calib.pkl", "rb") as f:
    calib = pickle.load(f)
mtx = calib["mtx"]
dist_original = calib["dist"]
dist = None
n = calib["n"]

R = np.load("/home/freitas/TCC/ender-laser-scanner/calib_data/v1_raspi/cam_to_printer.npy")

board, aruco_dict = load_board("board.pkl")

print(board.getChessboardSize())
print(board.getMarkerLength())
print(board.getSquareLength())

(16, 16)
9.375
12.5


In [5]:
n/np.linalg.norm(n[:3])
#n/np.linalg.norm(n[:3])/np.linalg.norm(n[:3])

array([ 2.30883819e-02,  8.61715312e-01, -5.06866499e-01,  8.83359698e+01])

In [6]:
R

array([[ 0.9985865 , -0.02415544,  0.04734473, -0.20928927],
       [ 0.00277792, -0.86582552, -0.50033835, -0.92947292],
       [ 0.05307817,  0.49976263, -0.86453457,  0.62204345]])

In [7]:
def load(path):
    info = {}
    data = []
    with open(join(path,"points.json")) as f:
        info = json.loads(f.read())

    for fig, point in info.items():
        image =  cv2.cvtColor(cv2.imread(join(path, fig)), cv2.COLOR_BGR2RGB)
        image = cv2.undistort(image, mtx, dist_original, None, mtx)
        data.append( (point, image) )

    return data

def load_itr(path):
    info = {}
    data = []
    with open(join(path,"points.json")) as f:
        info = json.loads(f.read())

    for fig, point in info.items():
        image =  cv2.cvtColor(cv2.imread(join(path, fig)), cv2.COLOR_BGR2RGB)
        image = cv2.undistort(image, mtx, dist_original, None, mtx)
        yield point, image

In [63]:
data = load(data_path+"low_exp")

points = [d[0] for d in data]
images = [d[1] for d in data]

KeyboardInterrupt: 

In [8]:
red_threshold = 20
def find_laser(image):
    blur = cv2.GaussianBlur(image, ksize=(5,5), sigmaX=1)
    centroids = co.column_centroids(co.red_contrast(blur),
                                    mask=co.red_contrast(blur) > red_threshold)

    # argmaxes = co.column_argmaxes(co.red_contrast(blur),
    #                                 mask=co.red_contrast(blur) > red_threshold)
    
    
                                    
    return centroids

In [9]:
image = data[8][1]

blur = cv2.GaussianBlur(image, ksize=(5,5), sigmaX=1)
centroids = co.column_centroids(co.red_contrast(blur),
                                mask=co.red_contrast(blur) > red_threshold)

argmaxes = co.column_argmaxes(co.red_contrast(blur),
                                mask=co.red_contrast(blur) > red_threshold)

fig, axs = plt.subplots(2,1, sharex=True)
rows = range(len(centroids))
axs[0].plot(centroids)
axs[0].plot(argmaxes)
axs[1].imshow(image)

In [9]:
from matplotlib import animation

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
# animation function.  This is called sequentially
imgplot = None
pointsplot = None
data_gen = load_itr(data_path + "low_exp")
point0, image0 = next(data_gen)
data_gen = load_itr(data_path + "low_exp")

def init():
    global imgplot, pointsplot
    imgplot = plt.imshow(image0, animated=True)
    pointsplot, = plt.plot(range(image0.shape[0]), "r.", animated=True)
    return pointsplot, imgplot


def laser_points_img_gen(i):
    global imgplot, pointsplot
    # Find board and laser
    # point, image =data[i]
    point, image = next(data_gen)

    # blur = cv2.GaussianBlur(image, ksize=(5,5), sigmaX=1)

    im = image
    centroids = find_laser(image)
    # centroids = co.column_argmaxes(co.red_contrast(image), mask=co.red_contrast(image)>50)
    p_centroids = np.stack(list((i, p, 1) for i,p in enumerate(centroids) if not np.isnan(p))).T

    # Update anim
    imgplot.set_data(image)
    pointsplot.set_data(p_centroids[0], p_centroids[1])
    return pointsplot, imgplot

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, laser_points_img_gen, init_func=init,
                               frames=200, interval=20, blit=True)

In [10]:
def ray_plane_intersect_vectorized(img_points, plane, cam_mtx):
	"""
	Calculate 3d-intersection between image rays defined by `img_ponts` and `cam_mtx`
	and a plane defined by the 4-vec `plane`
	@param img_points np.array with shape (3, N), N= number of points, homegeneous coords
	@param plane np.array with shape = (4,)
	@param cam_mtx np.array with shape = (3,3), intrinsic camera parameters
	@returns np.array with shape (4, N) with 3d points
	"""
	assert plane.shape == (4,)
	assert len(img_points.shape) == 2
	assert img_points.shape[0] == 3

	rays = np.linalg.inv(cam_mtx)@img_points
	p = plane[-1]
	n = plane[:3]
	out = (-p/(n@rays)) * rays
	out = np.vstack((out, np.ones((1, out.shape[-1])))) # homogeneous coords
	return out

In [11]:
import time
def triangulate_pts(image):
	centroids = find_laser(image)
	p_centroids = np.stack(list((i, p, 1) for i,p in enumerate(centroids) if not np.isnan(p))).T

	pts_3d = ray_plane_intersect_vectorized(p_centroids, n, mtx)
	return pts_3d
def chess_plane(rvec, tvec):
	# Find p
	n_chessboard = np.array([0, 0, 1])
	M,_ = cv2.Rodrigues(rvec)
	n3 = M@n_chessboard
	p = -1 * n3@tvec
	n = np.hstack((n3,p))
	return n

In [12]:
W = np.eye(4)
Xlist = []
XcList=[]
for point, image  in data:
    # 3d pts in camera frame
    pts_3d = triangulate_pts(image)
    # Extrinsic params
    # rot, jac = cv2.Rodrigues(rvec)
    # W[:3, -1] = tvec.flatten()
    # W[:3,:3] = rot

    # Concat points
    XcList.append(pts_3d)
    # Xlist.append(np.linalg.inv(W)@pts_3d)
# X = np.hstack(Xlist)

NameError: name 'data' is not defined

## Plot

In [13]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import importlib
import common as co
importlib.reload(co)

<module 'common' from '/home/freitas/TCC/ender-laser-scanner/notebook/common.py'>

### Profiles

In [17]:
import matplotlib.ticker as plticker

fig, ax = co.plot3d(proj_type="ortho")
for i, x in enumerate(Xlist[3:13]):
    x = x.copy()
    x = np.linalg.inv(cam2printer)@x
    x = x[:,x[0] > -10]
    x = x[:,x[0] < 60]
    ax.scatter(x[0], x[1], x[2], label=f"{3+i}")

#co.set_axes_equal(ax)
ax.set_xlabel("$X_w ~[mm]$")
ax.set_ylabel("$Y_w ~[mm]$")
ax.set_zlabel("$Z_w ~[mm]$")

#ax.invert_zaxis()

#ax.set_ylim(-0.5, i)
#ax.set_zlim(90, 100)
#ax.set_xlim(-60, 0)

#ax.xaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
# ax.yaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
# ax.zaxis.set_major_locator(plticker.MultipleLocator(base=1.0))

ax.legend(loc="upper right", ncol=2, title="Estação").set_draggable(True)
plt.title("Visualização 3D das Nuvens Parciais")
fig.tight_layout()

In [15]:
for i, x in enumerate(XcList):
    #x = x[:,x[2] < 110]
    plt.plot(x[0], x[2], "--.", label=f"{i}")

plt.gca().invert_yaxis()

plt.xlabel(r"$X_c~ [mm]$")
plt.ylabel(r"$Z_c~ [mm]$")
ax.legend(loc="best", ncol=2, title="Perfil ($i_p$)")
plt.grid()
plt.title("Projeção dos Perfis em XZ")
plt.legend().set_draggable(True)
plt.tight_layout()

NameError: name 'ax' is not defined

In [18]:
chosen_pics = slice(3,13)

data = load(data_path+"low_exp")
d = data[chosen_pics]

cols = 2
fig, axs = plt.subplots(len(d)//cols,cols, sharex=True, sharey=True)

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

for i, ( ax, (point, image) ) in enumerate( zip(axs.flatten(), d) ):

    # Axes
    ax.imshow(d[i][1])
    ax.axis("off")
    ax.set_title(f"Estação {3+i}", color=colors[i])

    centroids = find_laser(image)
    p_centroids = np.stack(list((i, p, 1) for i,p in enumerate(centroids) if not np.isnan(p))).T
    ax.plot(p_centroids[0], p_centroids[1], "g.", markersize=1)

# Reconstruct with `cam_to_printer`

In [14]:
R = np.load("/home/freitas/TCC/ender-laser-scanner/calib_data/v1_raspi/cam_to_printer.npy")
print(R)
p0, _ = next(load_itr(data_path + "low_exp"))
p0 = np.array(p0)
print(p0)

cam2printer = np.eye(4)
cam2printer[:3,:3] = R[:3,:3]
print(cam2printer)

[[ 0.9985865  -0.02415544  0.04734473 -0.20928927]
 [ 0.00277792 -0.86582552 -0.50033835 -0.92947292]
 [ 0.05307817  0.49976263 -0.86453457  0.62204345]]
[125.  90.  60.]
[[ 0.9985865  -0.02415544  0.04734473  0.        ]
 [ 0.00277792 -0.86582552 -0.50033835  0.        ]
 [ 0.05307817  0.49976263 -0.86453457  0.        ]
 [ 0.          0.          0.          1.        ]]


In [15]:
data_path="/home/freitas/TCC/ender-laser-scanner/pics/scan1_raspi/"

In [16]:
W = np.eye(4)
Xlist = []
XcList=[]

i=0
for point, image in co.progressbar(load_itr(data_path + "low_exp")):
    i+=1
    # if (i > 50):
    #     break
    # if (i < 50):
    #     continue
    # if (i >= 100):
    #     break
    # 3d pts in camera frame
    pts_3d = triangulate_pts(image)

    # Transformation from first point to this one (printer coords)
    # B = np.eye(4)
    # B[:3,3] = np.array(point) - p0

    A = np.eye(4)
    A[:3,3] = R[:3,:3]@(np.array(point) - p0)


    # Transformation from the first camera to this one
    # A = cam2printer@B@np.linalg.inv(cam2printer)

    # Concat points
    XcList.append(pts_3d)
    Xlist.append(A@pts_3d)
X = np.hstack(Xlist)

50it [00:39,  1.25it/s]


In [13]:
name = data_path.split("/")[-2]
print(name)
X = np.linalg.inv(cam2printer)@np.hstack(Xlist)
np.savetxt(f"clouds/{name}2.txt", np.round(X.T[:,:3],3), delimiter=",", fmt="%.3f")

scan1_raspi


In [27]:
for file_ind in range(1,11):
    data_path=f"/home/freitas/TCC/ender-laser-scanner/pics/scan_cold_{file_ind}/"

    Xlist = []

    for point, image in co.progressbar(load_itr(data_path + "low_exp")):
        pts_3d = triangulate_pts(image)
        A = np.eye(4)
        A[:3,3] = R[:3,:3]@(np.array(point) - p0)
        Xlist.append(A@pts_3d)
    X = np.linalg.inv(cam2printer)@np.hstack(Xlist)
    name = data_path.split("/")[-2]
    print(f"{name} Done!")
    np.savetxt(f"clouds/{name}.txt", np.round(X.T[:,:3],3), delimiter=",", fmt="%.5f")

121it [01:49,  1.10it/s]


scan_cold_1 Done!


121it [01:41,  1.19it/s]


scan_cold_2 Done!


121it [01:40,  1.21it/s]


...

scan_cold_8 Done!


121it [01:30,  1.34it/s]


scan_cold_9 Done!


121it [01:29,  1.35it/s]


scan_cold_10 Done!


In [43]:
import matplotlib.ticker as plticker

fig, ax = co.plot3d(proj_type="ortho")
for i, x in enumerate(Xlist):
    #x = np.linalg.inv(cam2printer)@x
    ax.scatter(x[0], x[1], x[2], label=f"{i}")

#ax.set_ylim(-0.5, i)
#ax.set_zlim(90, 100)
#ax.set_xlim(-60, 0)

#ax.xaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
#ax.yaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
#ax.zaxis.set_major_locator(plticker.MultipleLocator(base=1.0))

fig.tight_layout()

In [64]:
p0

array([125.,  90.,  60.])

In [14]:
import matplotlib.ticker as plticker

#fig, ax = co.plot3d(proj_type="ortho")
fig = plt.figure()
ax = fig.add_subplot(211, projection="3d")
ax2 = fig.add_subplot(212)
X_printer = X.copy()#np.linalg.inv(cam2printer)@X
X_printer[0] = X_printer[0] + p0[0]
X_printer[1] = X_printer[1] + p0[1]
X_printer[2] = X_printer[2] + p0[2]
    #x = np.linalg.inv(cam2printer)@x
# X_printer = X_printer[:,X_printer[2] > -144]
# X_printer = X_printer[:,X_printer[1] > 92]
# X_printer = X_printer[:,X_printer[1] < 108]
X_printer = X_printer[:,X_printer[0] > +p0[0]+-10]
subsample = 1
p=ax.scatter(X_printer[0, ::subsample], X_printer[1, ::subsample], X_printer[2, ::subsample], c=X_printer[2,::subsample], cmap="turbo")
fig.colorbar(p)

ax._axis3don = False
X_printer = X_printer[:,X_printer[1] > +p0[1]+97]
X_printer = X_printer[:,X_printer[1] < +p0[1]+110]
ax2.scatter(X_printer[0, ::subsample], X_printer[2, ::subsample])

ax2.xaxis.set_minor_locator(plt.MultipleLocator(1))
ax2.yaxis.set_minor_locator(plt.MultipleLocator(1))
ax2.xaxis.set_major_locator(plt.MultipleLocator(5))
ax2.yaxis.set_major_locator(plt.MultipleLocator(5))
ax2.set_ylabel("$z_W$")
ax2.set_xlabel("$x_W$")
#ax2.set_aspect("equal")
ax2.grid(True, which="Both")

fig.suptitle("Nuvem de Pontos da Peça A")



#ax.set_ylim(-0.5, i)
#ax.set_zlim(90, 100)
#ax.set_xlim(-60, 0)

#ax.xaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
#ax.yaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
#ax.zaxis.set_major_locator(plticker.MultipleLocator(base=1.0))

co.set_axes_equal(ax)
#co.set_axes_equal(ax2)

fig.tight_layout()

/tmp/ipykernel_31250/2274099290.py:18: MatplotlibDeprecationWarning: Starting from Matplotlib 3.6, colorbar() will steal space from the mappable's axes, rather than from the current axes, to place the colorbar.  To silence this warning, explicitly pass the 'ax' argument to colorbar().
  fig.colorbar(p)


In [58]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#region = X[2] > -9999
ax.plot_trisurf(X_printer[0,::subsample],X_printer[1,::subsample], X_printer[2,::subsample], color="gray")
co.set_axes_equal(ax)

In [37]:
plt.figure()
plt.scatter(X_printer[0], X_printer[2], s=2)

# Output